# VacationPy
----


In [16]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [17]:
# File to Load
output_data_file = "output_data/city_data.csv"

# Read data file and store into Pandas Data Frame
cityData_df = pd.read_csv(output_data_file)
cityData_df.head()

,City,Latitude,Longitude,Humidity,Temperature,Clouds,Wind
0,Kismayo,-0.3582,42.5454,82,25.79,9,6.02
1,Gīdolē,5.6500,37.3667,69,15.66,48,0.81
2,Busselton,-33.6500,115.3333,54,23.38,40,3.81
3,Havøysund,70.9963,24.6622,60,1.49,100,14.72
4,Zeerust,-25.5369,26.0751,78,19.45,99,5.06


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [18]:
gmaps.configure(api_key=g_key)

In [19]:
locations = cityData_df[["Latitude", "Longitude"]]
weights = cityData_df["Humidity"]

In [48]:
fig = gmaps.figure()

fig

Figure(layout=FigureLayout(height='420px'))

In [50]:
heatmap_layer = gmaps.heatmap_layer(locations, weights=weights)
heatmap_layer.max_intensity = 200
heatmap_layer.point_radius = 5
fig.add_layer(heatmap_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [42]:
perfect_weather_df = cityData_df.loc[(cityData_df["Temperature"] < 26.66) & (cityData_df["Temperature"] > 12.77) \
                                    & (cityData_df["Wind"] < 10) \
                                    & (cityData_df["Clouds"] == 0)]
perfect_weather_df.dropna()
perfect_weather_df

,City,Latitude,Longitude,Humidity,Temperature,Clouds,Wind
7,Punta Arenas,-53.1500,-70.9167,54,16.01,0,4.63
17,Port Elizabeth,-33.9180,25.5701,77,21.17,0,4.12
23,Los Llanos de Aridane,28.6585,-17.9182,41,18.96,0,1.79
28,Xucheng,20.3500,110.1288,72,16.39,0,3.59
49,Bilma,18.6853,12.9164,28,14.34,0,3.55
55,Pacific Grove,36.6177,-121.9166,75,18.65,0,1.34
64,Geraldton,-28.7667,114.6000,49,20.23,0,3.09
78,Lebu,-37.6167,-73.6500,63,16.16,0,4.96
120,Najrān,17.4924,44.1277,60,18.36,0,0.56
145,Cape Town,-33.9258,18.4232,69,19.49,0,1.34


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [44]:
hotel_df = pd.DataFrame(perfect_weather_df)
hotel_df["Hotel"] = ""
hotel_df.head()

,City,Latitude,Longitude,Humidity,Temperature,Clouds,Wind,Hotel
7,Punta Arenas,-53.1500,-70.9167,54,16.01,0,4.63,
17,Port Elizabeth,-33.9180,25.5701,77,21.17,0,4.12,
23,Los Llanos de Aridane,28.6585,-17.9182,41,18.96,0,1.79,
28,Xucheng,20.3500,110.1288,72,16.39,0,3.59,
49,Bilma,18.6853,12.9164,28,14.34,0,3.55,


In [ ]:
url = 
query = f""
response = requests.get(query)

In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
# Add marker layer ontop of heat map


# Display figure
